In [1]:
import pandas as pd

# list of file names to read in
# file_names = [
#               'PNEOA_15193.log', 'PNEOA1_15178.log', 'PNEOA2_15179.log', 'PNEOA3_15180.log', 'PNEOA4_15181.log', 'PNEOA5_18100.log', 'PNEOA6_18100.log',
#               'PNEOC_15190.log', 'PNEOC1_15184.log', 'PNEOC2_15185.log', 'PNEOC3_15186.log', 'PNEOC4_15187.log', 'PNEOC5_15188.log', 'PNEOC6_15189.log',
#               'PNEOE_15192.log', 'PNEOE1_15166.log', 'PNEOE2_15167.log', 'PNEOE3_15168.log', 'PNEOE4_15169.log', 'PNEOE5_15170.log', 'PNEOE6_15171.log',
#               'PNEON_15194.log', 'PNEON1_15160.log', 'PNEON2_15161.log', 'PNEON3_15162.log', 'PNEON4_15163.log', 'PNEON5_15164.log', 'PNEON6_15165.log',
#               'PNEOO_15191.log', 'PNEOO1_18102.log', 'PNEOO2_15173.log', 'PNEOO3_15174.log', 'PNEOO4_15175.log', 'PNEOO5_15176.log', 'PNEOO6_15177.log',
#             ]

# populate file_names with the names of each file in the output_logs folder
import os
file_names = []
file_names_truncated = []
for file in os.listdir('output_logs_Middle_Quartiles'):
# for file in os.listdir('output_logs'):
    file_names.append(file)
    file = file[0:(len(file)-9)]
    print("Name",file)
    file_names_truncated.append(file)
    
    
# remove the last item in the list (the .DS_Store file)
file_names_truncated.pop(0)
file_names.pop(0)
# file_names.pop(len(file_names) - 1)

# create a dataframe of dataframes to hold the results for each file
file_dataframes = []
for fileName in file_names_truncated:
    df = pd.DataFrame(columns=['fileName', 'r2', 'mse'])
    df['fileName'] = [fileName] * 7
    file_dataframes.append(df)


# loop through each file name in file_names
counter = 0
for fileName in file_names:   

    # open file and read lines
    open_file = open(fileName, 'r')
    read_file = open_file.readlines()

    r2_list = []
    mse_list = []

    for line in read_file:
        # if line includes r2, grab following characters until the next "," and store in r2
        if '\'r2' in line:
            r2 = line[line.find('\'r2\': ') + 6:line.find(',', line.find('\'r2\': '))]
            r2_list.append(r2)
        # if line includes mse, grab following characters until the next "," and store in rmse
        if '\'mse' in line:
            mse = line[line.find('\'mse\': ') + 7:line.find(',', line.find('\'mse\': '))]
            mse_list.append(mse)

    # add r2 and mse lists to dataframe
    file_dataframes[counter]['r2'] = r2_list
    file_dataframes[counter]['mse'] = mse_list

    counter = counter + 1


Name .ipynb_ch
Name PNEOA4_Mid_1
Name PNEOA3_Mid_1
Name PNEOA1_Mid_1
Name PNEOA2_Mid_1
Name PNEOA_Mid_1
Name PNEOA5_Mid_1
Name PNEOA6_Mid_1


In [2]:
# print list of dataframes
for df in file_dataframes:
    print(df)

       fileName                      r2                 mse
0  PNEOA4_Mid_1   -0.029377386099276315  1.0347860408295404
1  PNEOA4_Mid_1  0.00010798825257996025  1.1742535059287895
2  PNEOA4_Mid_1   -0.016721296299668564   1.067984368298503
3  PNEOA4_Mid_1   -0.004279577861522554   0.895601634577159
4  PNEOA4_Mid_1    -0.01091526189834724  1.0702883374180048
5  PNEOA4_Mid_1  -0.0036404006009396195  1.0036404045945824
6  PNEOA4_Mid_1  -0.0036404006009396195  1.0036404045945824
       fileName                     r2                 mse
0  PNEOA3_Mid_1   -0.02438431196723667  1.0380494689104587
1  PNEOA3_Mid_1  -0.002781801387617744   1.109988008986234
2  PNEOA3_Mid_1  0.0016740625227104067  1.1062205740030007
3  PNEOA3_Mid_1  -0.008248679885444421  0.9607738066149826
4  PNEOA3_Mid_1   -0.16067750634277544  1.2499890416010981
5  PNEOA3_Mid_1   -0.04490057104208467  1.0449005730544962
6  PNEOA3_Mid_1   -0.04490057104208467  1.0449005730544962
       fileName                     r2          

In [3]:
# create a single dataframe from the list of dataframes
final_df = pd.DataFrame(columns=['fileName', 'r2_train_max', 'r2_train_avg', 'r2_test', 'rmse_train_min', 'rmse_train_avg', 'rmse_test', 'learning_rate', 'epochs'])

fileName_list = []
r2_train_max_list = []
r2_train_avg_list = []
r2_test_list = []
rmse_train_min_list = []
rmse_train_avg_list = []
rmse_test_list = []

learning_rate = '2e-5' 
epochs = '30_batchSize_48:24_16point_8gpu'

# loop through each dataframe in file_dataframes
for file_df in file_dataframes:
    # get the file name
    fileName = file_df['fileName'][0]

    # split train and test data
    r2_train = file_df['r2'][0:5]
    r2_test = file_df['r2'][6]
    rmse_train = file_df['mse'][0:5]
    rmse_test = file_df['mse'][6]

    # convert lists to floats
    r2_train = [float(i) for i in r2_train]
    r2_test = float(r2_test)
    rmse_train = [float(i) for i in rmse_train]
    rmse_test = float(rmse_test)

    # get maxes and averages
    r2_train_max = max(r2_train)
    r2_train_avg = sum(r2_train) / len(r2_train)
    rmse_train_min = min(rmse_train)
    rmse_train_avg = sum(rmse_train) / len(rmse_train)

    # add the data to the dataframe
    fileName_list.append(fileName)
    r2_train_max_list.append(r2_train_max)
    r2_train_avg_list.append(r2_train_avg)
    r2_test_list.append(r2_test)
    rmse_train_min_list.append(rmse_train_min)
    rmse_train_avg_list.append(rmse_train_avg)
    rmse_test_list.append(rmse_test)

# add the lists to the dataframe
final_df['fileName'] = fileName_list
final_df['r2_train_max'] = r2_train_max_list
final_df['r2_train_avg'] = r2_train_avg_list
final_df['r2_test'] = r2_test_list
final_df['rmse_train_min'] = rmse_train_min_list
final_df['rmse_train_avg'] = rmse_train_avg_list
final_df['rmse_test'] = rmse_test_list
final_df['learning_rate'] = learning_rate
final_df['epochs'] = epochs

In [4]:
print(final_df)

       fileName  r2_train_max  r2_train_avg   r2_test  rmse_train_min  \
0  PNEOA4_Mid_1      0.000108     -0.012237 -0.003640        0.895602   
1  PNEOA3_Mid_1      0.001674     -0.038884 -0.044901        0.960774   
2  PNEOA1_Mid_1     -0.000831     -0.121492 -0.199942        0.986229   
3  PNEOA2_Mid_1     -0.001291     -0.048884 -0.002314        0.920048   
4   PNEOA_Mid_1     -0.002304     -0.019087 -0.009614        0.941339   
5  PNEOA5_Mid_1      0.011237     -0.021242  0.010957        0.857929   
6  PNEOA6_Mid_1     -0.000018     -0.045151 -0.021684        0.740955   

   rmse_train_avg  rmse_test learning_rate                           epochs  
0        1.048583   1.003640          2e-5  30_batchSize_48:24_16point_8gpu  
1        1.093004   1.044901          2e-5  30_batchSize_48:24_16point_8gpu  
2        1.145871   1.199942          2e-5  30_batchSize_48:24_16point_8gpu  
3        1.112065   1.002314          2e-5  30_batchSize_48:24_16point_8gpu  
4        1.081146   1.009

In [5]:
# export dataframe to excel
epochs = str(epochs)
# output_xl = 'Psych_AI_Output_'+learning_rate+'_'+epochs+'3.xlsx'
output_xl = 'Psych_AI_Output_Middle_Quartiles_80_percent_'+learning_rate+'_'+epochs+'2.xlsx'
final_df.to_excel(output_xl)